In [1]:
!pip install streamlit
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 4.2 MB/s eta 0:00:00


In [2]:
import streamlit as st


# Інтерфейс Streamlit
st.title("Експериментування з параметрами моделі")
st.write("Змініть параметри моделі та запустіть тренування.")

# Параметри моделі
units = st.slider("Кількість LSTM-юнітів", min_value=16, max_value=256, step=16, value=64)
embedding_dim = st.slider("Розмір вбудовування (embedding_dim)", min_value=16, max_value=256, step=16, value=64)
maxlen = st.slider("Максимальна довжина тексту (maxlen)", min_value=50, max_value=500, step=50, value=100)
epochs = st.slider("Кількість епох (epochs)", min_value=1, max_value=20, step=1, value=5)
batch_size = st.slider("Розмір пакету (batch_size)", min_value=16, max_value=128, step=16, value=32)

# Кнопка запуску навчання
if st.button("Запустити навчання"):
    st.write("Тренування моделі...")
    accuracy, fig = train_model(units, embedding_dim, maxlen, epochs, batch_size)
    st.write(f"Точність на тестовому наборі: {accuracy:.4f}")
    st.pyplot(fig)

2024-12-16 18:39:50.215 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 18:39:50.582 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-12-16 18:39:50.591 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 18:39:50.600 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 18:39:50.603 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 18:39:50.606 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 18:39:50.613 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-16 18:39:50.620 Thread 'MainThread': mi

In [ ]:
import streamlit as st
import subprocess
import threading
import time

from pyngrok import ngrok

# Сохраните скрипт Streamlit
with open('app.py', 'w') as f:
    f.write('''import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import joblib

def load_and_prepare_data():
    # Загрузка данных
    file_path = '/content/drive/MyDrive/spam2.csv'
    df = pd.read_csv(file_path, encoding='latin1')

    # Очистка данных
    df.drop(columns=['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], inplace=True)
    df = df.rename(columns={'v1': 'label', 'v2': 'text'})

    # Кодирование меток
    le = LabelEncoder()
    df['label'] = le.fit_transform(df['label'])

    # Разделение данных
    X = df['text']
    y = df['label']

    # Разделение на train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

def prepare_sequences(X_train, X_test, maxlen=100, num_words=5000):
    # Токенизация
    tokenizer = Tokenizer(num_words=num_words, oov_token="<OOV>")
    tokenizer.fit_on_texts(X_train)

    # Сохранение токенизатора
    joblib.dump(tokenizer, 'tokenizer.joblib')

    # Преобразование в последовательности
    X_train_seq = tokenizer.texts_to_sequences(X_train)
    X_test_seq = tokenizer.texts_to_sequences(X_test)

    # Паддинг
    X_train_pad = pad_sequences(X_train_seq, maxlen=maxlen, padding='post', truncating='post')
    X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen, padding='post', truncating='post')

    return X_train_pad, X_test_pad

def train_model(X_train_pad, y_train, X_test_pad, y_test, units=64, embedding_dim=64, maxlen=100, epochs=5, batch_size=32):
    # Создание модели с параметрами
    model = models.Sequential([
        layers.Embedding(input_dim=5000, output_dim=embedding_dim, input_length=maxlen),
        layers.LSTM(units, return_sequences=True),
        layers.GlobalMaxPooling1D(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')
    ])

    # Компиляция модели
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Обучение модели
    history = model.fit(
        X_train_pad, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_split=0.2,
        verbose=0
    )

    # Оценка модели
    loss, accuracy = model.evaluate(X_test_pad, y_test, verbose=0)

    # Сохранение модели
    model.save('spam_classification_model.h5')

    # Построение графика точности
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend()

    return accuracy, plt

def predict_message(message, maxlen=100):
    # Загрузка токенизатора и модели
    tokenizer = joblib.load('tokenizer.joblib')
    model = tf.keras.models.load_model('spam_classification_model.h5')

    # Токенизация и паддинг сообщения
    message_seq = tokenizer.texts_to_sequences([message])
    message_pad = pad_sequences(message_seq, maxlen=maxlen, padding='post', truncating='post')

    # Прогнозирование
    prediction = model.predict(message_pad)[0][0]
    return "Спам" if prediction > 0.5 else "Не спам", prediction

def main():
    st.title("Классификация спам-сообщений")

    # Параметры модели
    st.sidebar.header("Параметры модели")
    units = st.sidebar.slider("Количество LSTM-юнитов", min_value=16, max_value=256, step=16, value=64)
    embedding_dim = st.sidebar.slider("Размер вложения", min_value=16, max_value=256, step=16, value=64)
    maxlen = st.sidebar.slider("Максимальная длина текста", min_value=50, max_value=500, step=50, value=100)
    epochs = st.sidebar.slider("Количество эпох", min_value=1, max_value=20, step=1, value=5)
    batch_size = st.sidebar.slider("Размер пакета", min_value=16, max_value=128, step=16, value=32)

    if st.button("Запустить обучение"):
        try:
            # Загрузка и подготовка данных
            X_train, X_test, y_train, y_test = load_and_prepare_data()

            # Подготовка последовательностей
            X_train_pad, X_test_pad = prepare_sequences(X_train, X_test, maxlen=maxlen)

            # Обучение модели
            accuracy, fig = train_model(
                X_train_pad, y_train,
                X_test_pad, y_test,
                units=units,
                embedding_dim=embedding_dim,
                maxlen=maxlen,
                epochs=epochs,
                batch_size=batch_size
            )

            # Отображение результатов
            st.write(f"Точность на тестовом наборе: {accuracy:.4f}")
            st.pyplot(fig)

            st.success("Модель успешно обучена и сохранена!")

        except Exception as e:
            st.error(f"Ошибка при обучении: {str(e)}")

    st.header("Проверить сообщение")
    message = st.text_input("Введите сообщение для проверки:")
    if st.button("Проверить сообщение"):
        if message.strip():
            label, confidence = predict_message(message)
            st.write(f"Результат: **{label}** (уверенность: {confidence:.2f})")
        else:
            st.error("Введите сообщение для проверки!")

# Запуск приложения
if __name__ == "__main__":
    main()
''')

# Функция для запуска Streamlit
def run_streamlit():
    subprocess.run(["streamlit", "run", "app.py"])

# Создаем и запускаем поток для Streamlit
streamlit_thread = threading.Thread(target=run_streamlit)
streamlit_thread.start()

# Небольшая задержка для запуска Streamlit
time.sleep(5)

# Настройка ngrok
ngrok.set_auth_token('2qET4QwGI2E5GRFX4WK7RYelD2q_6AhGNYrZooZx1skoHFCzj')

# Создаем туннель
tunnel = ngrok.connect(8501)
print(f"Публичный URL: {tunnel}")

# Ожидание для поддержания туннеля открытым
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    ngrok.kill()

Публичный URL: NgrokTunnel: "https://393f-34-23-99-199.ngrok-free.app" -> "http://localhost:8501"
